In [ ]:
# Install necessary packages
%pip install -r requirements.txt  # Ensure you have the required packages

In [ ]:
# Import required libraries
import pandas as pd
import boto3
import json

In [ ]:
# Configuration Section
# ---------------------
config = {
    'aws_region': 'ap-south-1', #your aws region
    'aws_access_key_id': 'your-access-key-id',
    'aws_secret_access_key': 'your-secret-access-key',
    'model_id': 'your-model-id', #eg: anthropic.claude-3-sonnet-20240229-v1:0
    'input_file': 'input_food_data.xlsx', # upload your file in source folder  eg; food_data.xlsx
    'output_file': 'updated_food_data.xlsx',
    'column_name': 'Shrt_Desc', # column on which you want to perform enhancements
    'prompt_template': 'Provide a concise and descriptive title for the following food item. Only provide the title, without any additional text: {short_desc}' # promt
}


In [ ]:
# Initialize the boto3 client for Bedrock
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=config['aws_region'],
    aws_access_key_id=config['aws_access_key_id'],
    aws_secret_access_key=config['aws_secret_access_key'],
)

In [ ]:
# Load data from an Excel file
data = pd.read_excel(config['input_file'])

# Select the relevant column
data_sample = data[config['column_name']]
print(data_sample.head())  # Print the first few rows for verification

In [ ]:
# Function to generate a title using Bedrock
def generate_detailed_description(short_desc):
    try:
        prompt = config['prompt_template'].format(short_desc=short_desc)
        response = bedrock.invoke_model(
            modelId=config['model_id'],
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 60,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            }),
            contentType="application/json",
            accept="application/json"
        )

        # Parse the response
        response_body = json.loads(response['body'].read())
        response_text = response_body['content'][0]['text'].strip()

        return response_text

    except Exception as e:
        print(f"Error in generate_description: {str(e)}")
        return "Error generating description"

In [ ]:
# Apply the function to the selected column
data['Title'] = data_sample.apply(generate_detailed_description)

In [ ]:
print(data.head())  # Print the first few rows of the updated data for verification

# Save the updated data to a new Excel file
data.to_excel(config['output_file'], index=False)